In [1]:
#!pip install scipy
#!pip install keras

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


In [3]:
# ファイルの読み込み
path = './kazeno_matasaburo.txt'

In [4]:
bindata = open(path, 'rb').read()

In [5]:
# バイナリデータのデコード
text = bindata.decode('shift_jis')

In [6]:
print('Size of text:', len(text))

Size of text: 33106


In [7]:
# set関数でユニークは集合を作成
# そのあとのSortのために、集合データをlist化
chars = sorted(list(set(text)))
chars

['\n',
 '\r',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '_',
 'a',
 'f',
 'g',
 'h',
 'i',
 'j',
 'n',
 'o',
 'p',
 'r',
 't',
 'w',
 'z',
 '×',
 '−',
 '\u3000',
 '、',
 '。',
 '々',
 '《',
 '》',
 '「',
 '」',
 '『',
 '』',
 '【',
 '】',
 'あ',
 'い',
 'う',
 'え',
 'お',
 'か',
 'が',
 'き',
 'ぎ',
 'く',
 'ぐ',
 'け',
 'げ',
 'こ',
 'ご',
 'さ',
 'ざ',
 'し',
 'じ',
 'す',
 'ず',
 'せ',
 'ぜ',
 'そ',
 'ぞ',
 'た',
 'だ',
 'ち',
 'ぢ',
 'っ',
 'つ',
 'づ',
 'て',
 'で',
 'と',
 'ど',
 'な',
 'に',
 'ぬ',
 'ね',
 'の',
 'は',
 'ば',
 'ぱ',
 'ひ',
 'び',
 'ぴ',
 'ふ',
 'ぶ',
 'ぷ',
 'へ',
 'べ',
 'ぺ',
 'ほ',
 'ぼ',
 'ぽ',
 'ま',
 'み',
 'む',
 'め',
 'も',
 'ゃ',
 'や',
 'ゅ',
 'ゆ',
 'ょ',
 'よ',
 'ら',
 'り',
 'る',
 'れ',
 'ろ',
 'わ',
 'を',
 'ん',
 'ァ',
 'ア',
 'ィ',
 'イ',
 'ウ',
 'カ',
 'ガ',
 'キ',
 'ギ',
 'ク',
 'ケ',
 'コ',
 'サ',
 'シ',
 'ジ',
 'ス',
 'タ',
 'チ',
 'ッ',
 'ツ',
 'テ',
 'デ',
 'ト',
 'ド',
 'ナ',
 'ニ',
 'ネ',
 'ハ',
 'バ',
 'パ',
 'ビ',
 'フ',
 'ブ',
 'プ',
 'ペ',
 'ホ',
 'ボ',
 'ポ',
 'マ',
 'モ',
 'ャ',
 'ョ',
 'ラ',
 'リ',
 'ル',
 'ロ',
 'ン

In [8]:
print('Total chars:', len(chars))

Total chars: 834


In [9]:
# 文字と、indexの配列を作成する
# 
char_indices = dict((c, i) for i, c in enumerate(chars))
char_indices

{'\n': 0,
 '\r': 1,
 '-': 2,
 '.': 3,
 '/': 4,
 '0': 5,
 '1': 6,
 '2': 7,
 '3': 8,
 '4': 9,
 '5': 10,
 '6': 11,
 '7': 12,
 '8': 13,
 '9': 14,
 ':': 15,
 '_': 16,
 'a': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'n': 23,
 'o': 24,
 'p': 25,
 'r': 26,
 't': 27,
 'w': 28,
 'z': 29,
 '×': 30,
 '−': 31,
 '\u3000': 32,
 '、': 33,
 '。': 34,
 '々': 35,
 '《': 36,
 '》': 37,
 '「': 38,
 '」': 39,
 '『': 40,
 '』': 41,
 '【': 42,
 '】': 43,
 'あ': 44,
 'い': 45,
 'う': 46,
 'え': 47,
 'お': 48,
 'か': 49,
 'が': 50,
 'き': 51,
 'ぎ': 52,
 'く': 53,
 'ぐ': 54,
 'け': 55,
 'げ': 56,
 'こ': 57,
 'ご': 58,
 'さ': 59,
 'ざ': 60,
 'し': 61,
 'じ': 62,
 'す': 63,
 'ず': 64,
 'せ': 65,
 'ぜ': 66,
 'そ': 67,
 'ぞ': 68,
 'た': 69,
 'だ': 70,
 'ち': 71,
 'ぢ': 72,
 'っ': 73,
 'つ': 74,
 'づ': 75,
 'て': 76,
 'で': 77,
 'と': 78,
 'ど': 79,
 'な': 80,
 'に': 81,
 'ぬ': 82,
 'ね': 83,
 'の': 84,
 'は': 85,
 'ば': 86,
 'ぱ': 87,
 'ひ': 88,
 'び': 89,
 'ぴ': 90,
 'ふ': 91,
 'ぶ': 92,
 'ぷ': 93,
 'へ': 94,
 'べ': 95,
 'ぺ': 96,
 'ほ': 97,
 'ぼ': 98,
 'ぽ': 99,
 'ま

In [10]:
# indexから文字が引ける(上記の逆バージョン)
indices_char = dict((i, c) for i, c in enumerate(chars))
indices_char

{0: '\n',
 1: '\r',
 2: '-',
 3: '.',
 4: '/',
 5: '0',
 6: '1',
 7: '2',
 8: '3',
 9: '4',
 10: '5',
 11: '6',
 12: '7',
 13: '8',
 14: '9',
 15: ':',
 16: '_',
 17: 'a',
 18: 'f',
 19: 'g',
 20: 'h',
 21: 'i',
 22: 'j',
 23: 'n',
 24: 'o',
 25: 'p',
 26: 'r',
 27: 't',
 28: 'w',
 29: 'z',
 30: '×',
 31: '−',
 32: '\u3000',
 33: '、',
 34: '。',
 35: '々',
 36: '《',
 37: '》',
 38: '「',
 39: '」',
 40: '『',
 41: '』',
 42: '【',
 43: '】',
 44: 'あ',
 45: 'い',
 46: 'う',
 47: 'え',
 48: 'お',
 49: 'か',
 50: 'が',
 51: 'き',
 52: 'ぎ',
 53: 'く',
 54: 'ぐ',
 55: 'け',
 56: 'げ',
 57: 'こ',
 58: 'ご',
 59: 'さ',
 60: 'ざ',
 61: 'し',
 62: 'じ',
 63: 'す',
 64: 'ず',
 65: 'せ',
 66: 'ぜ',
 67: 'そ',
 68: 'ぞ',
 69: 'た',
 70: 'だ',
 71: 'ち',
 72: 'ぢ',
 73: 'っ',
 74: 'つ',
 75: 'づ',
 76: 'て',
 77: 'で',
 78: 'と',
 79: 'ど',
 80: 'な',
 81: 'に',
 82: 'ぬ',
 83: 'ね',
 84: 'の',
 85: 'は',
 86: 'ば',
 87: 'ぱ',
 88: 'ひ',
 89: 'び',
 90: 'ぴ',
 91: 'ふ',
 92: 'ぶ',
 93: 'ぷ',
 94: 'へ',
 95: 'べ',
 96: 'ぺ',
 97: 'ほ',
 98: 'ぼ',
 99: 'ぽ',
 10

In [11]:
maxlen = 40
step = 3
sentences = []
next_chars = []

In [12]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

In [13]:
print('Number of sentences:', len(sentences))

Number of sentences: 11022


In [14]:
# テキストのベクトル化
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [15]:
# モデルを定義する
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))

In [16]:
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [17]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [18]:
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('繰り返し回数:', iteration)
    model.fit(X, y, batch_size=128, epochs=1)
    
    start_index = random.randint(0, len(text)-maxlen-1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----diversity', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Seedを生成しました:' + sentence)
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=9)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
繰り返し回数: 1
Epoch 1/1
11022/11022 [==============================] - 34s 3ms/step - loss: 4.6101

----diversity 0.2
----- Seedを生成しました:
　そこで嘉助は、一生懸命それをつけて行きました。
　ところがその跡のような

　そこで嘉助は、一生懸命それをつけて行きました。
　ところがその跡のようなららいた。
　ああはって、はんんなはらんんなはらって、んんんはらいた。
　あ、はなんんなはらあいた。
　あんなは、、
　あんんながら、のうのうにはいました。
　ああいました。
　ああは、のいた。」
「あいんなはらいました。
　ああいました。
　ああいて、とんながらって、んんはんんなした。
　あははって、んんんないた。
　あんんんならた。」
　あんははんんんのした。
　あんははら、んんんながら、んんながら、それてらいました。
　三郎は、んなんないた。」
　あんんんうがら、んんながらいた。」
　ああいました。
　あんははいた。」
　ああって、、れなら、
　あんはういました。
　あんんなました。
　ああいて、とんんなはらんなはは、んんんんなはは、」とはんんはに、んんないた。
　あんんのうとがら、のはらうのうとないた。
　あはあいた。
　あんんはりた

----diversity 0.5
----- Seedを生成しました:
　そこで嘉助は、一生懸命それをつけて行きました。
　ところがその跡のような

　そこで嘉助は、一生懸命それをつけて行きました。
　ところがその跡のようながらなんいいた。」
　あいただらを、とのんう。」とがあらいたで、らむがました。
　みんはらだにらんいでいた。」
　なあないた。
　なあは一はらみんん鳴なって。」
　もんあのから川、とついた。」
　あもどんいた。
　あんだがいた。」郎ははからてらや、。
「さっないました。
　は鳴うう中かした。
　あはって、たん、いいした。
　ああくなら、ならました。
　あいてなました。
　があながいがほはいた。そははいてがた。
　郎うはどんなうへおらました。
　あって、たの付ですんんんて、らへげんらす、
「あ

　　　どっどど　どうう一郎どう立うう。い顔らゃががわふころがよ嘉助の味をそ葡《かに出した。」ぐいがい郎を水が水て風かの牧
は1手りへ《っちほんです子飲っ思らけまました。はひとみんで入時の野のに木の中にりは知っていせがろいト返濁だどろ。耕助は急中の馬るの葡来このこへ。寒なぞか。それからやいぐし方こさぞ玄草たほひぢゃでも小ょに折ったのをの山のでした。少しはれ　おでい速。いるがリ踊時んはこて《がり、波海土鳴は葉の忙のへ川《ら。そしろ顔じったのへ行きました。佐太郎が外いサをころを笑いち待ったて、耕助のうと立っス出この追へ、ぐ高数へ来て権ったのへは発ほどの鳴つ下にせって後棒がけへぞら悦悪いにおぎペちを人またへはってって、きのの春のほざお郎《じりりさならそのでり毛わにうったりま。」わざうがぶし登のざに代のうの子をよいっちのるのでは気ずの上人のを《うとううようの何へ急りきをのたってをつと叫びまめたよころやっげさんはげる革

--------------------------------------------------
繰り返し回数: 5
Epoch 1/1
11022/11022 [==============================] - 44s 4ms/step - loss: 2.5368

----diversity 0.2
----- Seedを生成しました:んと張って、そっちのほうをにらめていましたら、ちょうどそのとき、川上から、
「
んと張って、そっちのほうをにらめていましたら、ちょうどそのとき、川上から、
「そのは、
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　

があらわれました。嘉助はしばらく自分の目を疑って立ちどまっていましたが、やはりどばをふむげさすさありさまえてから、水をはい関も小の中へ行きました。寄じぶっとりし寒毛りに早らいたもり、そうの嘉助はすからの学校たえた今ぼふりせりました。
　ってここっちりおこうなくところこと近算とうっていていたり、喜時をラ下人に草きまるでしる顔、来たのです。高馬も耕助は通る抜おこしましたらそ令は学校へ赤いて。」
「嘉助勢もしばらくし行がし舞ふち谷のようとしていて出て来たいです。
　三郎は嘉助は一郎と叫びわが湯

ぶんなっチ」
「ろく枝　　どんくつえて来たののばしたた大りな来たばざさあり又三郎の向いつめの一人に本のわがあた集さや4丘下校《やぐ》ささささささささあ「南さ悪みゃさんが少し記」すたの》白い4やれて来て来たいで来た。をあいい一野目の木の上がり《子ど　こころきりに雑着手ああさこころ、》るかしろどく見っていた（手威あいいたがな羽聞《やぶぐ》さあきがさて、手にぐく算るッへありま

----diversity 1.2
----- Seedを生成しました:があらわれました。嘉助はしばらく自分の目を疑って立ちどまっていましたが、やはりど
があらわれました。嘉助はしばらく自分の目を疑って立ちどまっていましたが、やはりどは葉を音がら言いました。
「なも、んさいかお走して勉んでする。細叫びました。
「早う者近いたはッ栗《さり》ささささおささいっ畑、教の並二ト簿南たざ
「赤嘉対ぞ一令そのかジ起むて、葉べさん六目を指でしてしまじんだまけそれならがれつすた記げかよま大きな9じめこじみん根はそれからあり上ぶっぱっていまる一顔が着ていてそこ三へがず叫びました。
「そんでは、まの馬馬におおうえて下かっていませんでした。ぼ先生の枝きっとてが袴んやり一わはおり教室やひややり汗）こじめささい手り聞さつき横抜書ら息り、削ボ六二＃はぶか土だ《や》さささささ音雨波り手こささや五さささささりさ》ょゃあててぜ字手を手すみの課三二年生ら横のほうのでまたたでだなし、すくぼるのひ　バ待さありささ上げや屋払忙デ足り根五ささ六ト棒ささささぶさささ字）耕方さ4テトきたりませ白いw尾めは何る少に浮二つ行やをこざり陽やって五壇やり語字二ーせ二

-------------------------------------------------

んげん》さまの尾《お》っぱ持ちのようにすまし込んで、白いシャッポをかぶって、先生とにようにそれを見えい、その人は、
「あり、」ぐにおり、つだり、やっとしばれたしたいです。」
「なん、うだな。」と言いました。
「ああい、あいない。」と言いました。
　ところがく年生はじめました。それからまれでもなったよろの中のほうに、ござわるわとうとうとうさんが、たく、けたたたたもたもたくもすそのですから知らないで、たい、それは栗《やっていたよのであいた、もわが、ばたりしたのです。
　すると嘉助が言いました。
　ところが嘉助のおこっています。
「あり、ありり、又三郎おとうさん。」と言いながらにいるのです。
　ところが嘉助はするの三郎がったりしています。それからみんなははめれから寄れを見ないました。
　ところが嘉助のほうはおとうさんが、それからましたく、それからむれるのわしながらくまたの大きなにのほうに、いつわくわわわら、ながら手をはじめるのどうかわえていますとそけで、だ

----diversity 1.0
----- Seedを生成しました:んげん》さまの尾《お》っぱ持ちのようにすまし込んで、白いシャッポをかぶって、先生
んげん》さまの尾《お》っぱ持ちのようにすまし込んで、白いシャッポをかぶって、先生とにぐんとはみんなの言って行きました。一郎はしてぬるとかよわもあしなれかけたもです。
「一郎、言いなが。」と嘉助の木のぐるろきょうと小さんとにおら」
「また嘉って、先生のにつざいました。
　ところの嘉助はあるやわとすこうえて来て来たがこせです。
「上栗ありり、ひっは定るです。木のきなります。お耕てきました。
　みんなはここませんとしてもられるのふし気なよわの高くざっと三郎の分のうの屋のぼんでした。
　そらはいるのようだなったぶと飛が終ていましたしたばかどれどもけれどもそれど配どききききと歩と思きました。嘉助はしろほん走るの泳しなれからぼれを土そのときな、子のとひろ見えいのその下のほうではくくっとかんでしたな。」と言いました。
　みんなはこんがらそうどろからえるとうかうせん、しからないむとりまわしてようにしてったんでご大さながたがれば、
「たたいなので、ゃんと窓ガン教室の投あ

----diversity 1.2
----- Seedを生成しました:んげん》さまの尾《お》っぱ持ちのようにす

「ああひでりだ。なんだ、あるかあ、ござだな。」
　そのはじっとみんなはまるであとききながっていました。
　すると耕助はうしろしたく、
「おから、馬だ、やはだだ。」と言いました。
「ああいり、又三郎だだ。」
　その木のだんだんもすっといっています。
「あり、だだだだ。なんだ、ございます。」
「それから、
「なん、う、だだ。あるきが、
「あか、だだだ。だな。」
「又三郎吹きき。」
「それから、
「なん、う、だだ。あとか。」一郎はまるへんかりまって、そのかって来たのです。
「あい、出しなが。」
　一郎はまるであとかわいていました。
「さあ、こいでだ。ぐるだ。あるきない。」
　そのはじっとみんなの木の下の上がらまいで言いました。
「ああいひうだ。」と一郎はまるでをたって、しませました。
　そのうなだっぱりふきなきのようになったりまたた。そのうのとす。」
「たくなった。」

----diversity 0.5
----- Seedを生成しました:大急ぎでうちの中へはいると胸を一ぱいはって、息をふっと吹きました。
「ああひで
大急ぎでうちの中へはいると胸を一ぱいはって、息をふっと吹きました。
「ああひでりだ。だな。」
　そのはじめ、やっぱりかげてぺんなのたたのです。
　みんなはこの）そろでごろごろきました。
「おおいいなだ。おおいなあおりす。」
「なん、馬に、出したた。またた。その三郎がするのだよ。もあいます。」
「又三郎びって顔るであ。」
「わあいます。
　みんなはふりかえながら水したが、あとあちました。一郎は急いではいようにしてこっとそうです。
　みんなはひんどんとからえています。するのだんだももやったようになり、
「そっちよらに叫びまりが。」
「あり、だなだ。。」一郎はまる《かと》へならもうになり、馬の一人のとうになり言ったと、ぐむく、馬のこちにもっていたのです。
　そのうしろの耕助の草のかよろ立ちどました。
　みんなは、ばから」と言うながらんだのにぎをました。
　一郎はまるであときききますと、
「あか、このだ、このだだ。」
「そからな、このかした。一

----diversity 1.0
----- Seedを生成しました:大急ぎでうちの中へはいると胸を一ぱいはって、息をふっと吹きました。
「ああひで
大急ぎでうちの中へはいると胸を一ぱいはって、息をふっと吹きました。


...

「ああひう》だい。あと一郎の言いました。
「おおいいようお。」
「あかひひ吹《と》》風吹きていました。
　みんなは、じぶんどんどくくちにいました。
「さあ、そのだらだらみんな行きました。
　一郎がまるで顔のうになりました。そしてと三郎が、それからまいまってからなんなれか。ま　それからまい雨んでしました。
　みんなは、はじぶとを見てわくといちの顔をまたすけに行くなり、
「何きな、馬の、、、ラ、風な。風がら、　　　　　　　どうどんどんどんどんからつ木にやって、したったりました。
　耕助は木をぐるのでした。
　五年生の人の一郎がすまいたと、さは、一人のにはました。そして、ところを悦いでした。
　先生はちっとすぐあとこうでせなさん。それ

----diversity 1.0
----- Seedを生成しました:ませんでした。
　けれども木の向こう側に三郎のねずみいろのひじも見えていました
ませんでした。
　けれども木の向こう側に三郎のねずみいろのひじも見えていましたし、いたつものようにどれをもぼばたりして白耕助の顔を出しじしした。そしてと三郎に、をかざへ行きました。
　見ついもばしにいませんでしたが、向こうして一ぺんに二年たがらたた。」
「わあ、こののです。」
「ああ、うえ、この一人、ょう。」
「わあ、けれかあや。みんなはほんこんで見えて、
「うん。ああた笊《や》さぞ。」
　そのからだもよく権って鳴こました。
　びついど　くぐるそうでみんなう小さなくそうになりました。
「ああひう、だから、じめっとどう又三郎だ。みんな》はいればとおえまりして、
「まではそんとになから言いないな。」
「そがな又三郎、だっくりさらきちのだい。あとかありこさな。」
　それからみんない顔の木の下では栗《きり》の木のあるのどと立っていました。そしてがりの
「らめるのでして水で「た。するするだい。人も郎はまござって瓜るのでした。
　また手であいかちの子の

----diversity 1.2
----- Seedを生成しました:ませんでした。
　けれども木の向こう側に三郎のねずみいろのひじも見えていました
ませんでした。
　けれども木の向こう側に三郎のねずみいろのひじも見えていましたし、いつもも礼を何た言いました。
　が大発ぶのねむって来て窓窓はいたが、すぐむ水におりめに向こうで来たのです。
「学校飛ら

　みんなはなんだか、だんだかわあわぎな》にしてようにしてころのです。」
「みんな、うかどあこうではなようしてろ。うとうって一郎のにいさんの草の中でに二人のだぞ。」と言いました。
　みんなはこの上の野原のほうへいって行きました。
　嘉助はぶっとそのをんです。
「わあ、ちのようなようになり、そのにはいたようでああり、又三郎だっちのってしまいました。
　嘉助はやっと起ぶんが、
「空のくよいどてやっすの木のあり、ききのけばらくちを見ていました。
　すると嘉助はこんだとそのでした。
　先生は向こうでした。
　みんなはこのから、たいさんはまたがて、
「そから、
「あ

----diversity 0.5
----- Seedを生成しました:した。
　それからまわりがまっ蒼《さお》になって、ぐるぐる回り、とうとう深い草
した。
　それからまわりがまっ蒼《さお》になって、ぐるぐる回り、とうとう深い草の。けれどもどん歩れて来た人は急いませんでしたのでした。
　すると嘉助はとう五きながら嘉助はさるきききのわのわあて、ついはたたいの、の一郎が言いました。
「うん。」
　三郎はまた言いました。
「あいいや。」三郎のきの子の上の子の木はあいました。
　すると三郎は、どんだんました。
　すると嘉助はこんましたら、だいから、
「おうっこのだか。」と言いながらました。
　あいやどう。一郎は一郎よににわって、またがいてしてようでした。
　すると嘉助は立っしました。
　すると嘉助はうっとを見て、ててがたり、その青しろにそこでえて、
「耕助を見て、
「ああい。あのだい。ありり。あ。」と言いました。
　みんなはほんなうからすぐしてやっちのだのかしながらわれになりでにわくて、しろくました。
　そのうんと、時郎のところに立ちました。
「こびおがりあいひやだやや、やっとみんなに叫びま

----diversity 1.0
----- Seedを生成しました:した。
　それからまわりがまっ蒼《さお》になって、ぐるぐる回り、とうとう深い草
した。
　それからまわりがまっ蒼《さお》になって、ぐるぐる回り、とうとう深い草どどんけんとは一目立ちをると、きょところであいました。
　悪しな顔を見てすぐそ学校やっと運動場はあかりだといたのところがへ

底でうくらく道を見ていました。
「嘉助、うっしょにぐつけて小さなが、すから、

枝をひらぶのうしてのほうではなさんがいって来た人な。あのあ耕助まあた。じっは白いそっち少し行くが、とうような声で、ひとう小さながったな。」三二」二［はやっと運動場をひスげ「したが。空ば少しでゃいなんだ。みんなはなった。」それに」と言うながら車のところへ行きました。
変一郎がやったほうました。そしてすぐねくら知らっこうにじく霧のほうへ行きました。
　嘉助は歩るをもわえました。ぱしなもする

--------------------------------------------------
繰り返し回数: 50
Epoch 1/1
11022/11022 [==============================] - 50s 5ms/step - loss: 0.1810

----diversity 0.2
----- Seedを生成しました:した。
　みんなはしんとなってしまいました。だんだんみんな女の子たちも集まって
した。
　みんなはしんとなってしまいました。だんだんみんな女の子たちも集まって一年生の小さな生です。
「ああ、このだい。」と言ってまた言いました。
「行くなり、けれになってな。」と言いました。といがりがそのていました。
「はあい。」と言いました。
「おおいい。」と言いました。
「す。」
「お早あそれだよ。い。」と言いながらがいたように、
「そんだら、だそだら、だっ早草か。」と言いてれからました。
　一郎がやっとそのにました。それからまるでかって来て窓ガラスはのがったのように、言いました。すると佐太郎はにいながらまってそこにしまったのでした。
　すると一郎は木のどうからまれそのでした。と思ったようですたいうように向ころのです。みんなはどんどん歩いました。そしてときんのぼっとみんなの、おといるようでした。
　すると嘉助はこんどもすからないでした。それからまるでききました。
「この机の上がもいで言いない。」と言いました。
「す。」
「お早あそれか

----diversity 0.5
----- Seedを生成しました:した。
　みんなはしんとなってしまいました。だんだんみんな女の子たちも集まって
した。
　みんなはしんとなってしまいました。だんだんみんな女の子たちも集まって一年生にもみんないであ。」と言いました。
「しから、
「おおらいないおおり

「おまえないとおりしな塵ないで、　るっと風んどんどん鮒の歩ました。するといからわかあ、こうで字あり窓わあ、たいさないとうって一ぺん走しました。うんと言ってつこの前をけかといれ、したのです。すると一郎が、
「わあ、ちょうはすべす。」と言ってきました。
「嘉助、うのいつろで来た。」と言いました。
　めるとうんとは一郎とごはまって青へ行っています。
　そして一郎は、そのをとう走わあしたのです。
「でわあり、嘉助高わぼさんののようだ。」と言いました。礼ったらはみんな通ふかげししたのです。するといっもしたら。
　その

----diversity 1.2
----- Seedを生成しました:。
「さあ、流れて来るぞ。みんなとれ。」と一郎が言いました。まもなく耕助は小指
。
「さあ、流れて来るぞ。みんなとれ。」と一郎が言いました。まもなく耕助は小指ぐし思のな、、
「るるだなったよ。げなよい。」と言って来たんだ。」と言いました。
　それは嘉助は通をできました。するとかよ、大人はみんな追あと雑ご《かり》き寄へでびって、ぶさせゃふそれをちゃんとして、水を見てわ
「やっぱりだ栗---

れらでたいってらいいおこさな。一郎と言って、子の中をろきますとのさっきのいばむく気になりました思い大さなぎ方にしてようにしています。
　下のようど　どうどっここどんはども少し日な。何からに気年からするのになりました。一郎は急いませっぱりこめに、郎ののほうぞめいた。」と言いました。見てったの浮ああ知らつ鏡もぐく待っして見ますとなら、「まで石みんな見ていましたら、ちをするのからみんなしつから耕》音にはいて、あませかどしたなりました。下のくバ嘉助は
「あい字手で、又三郎だぞ。」
　その日しがらその前の戻ら一明ちのここをこん　　　　　　　　　鳴っ白く

--------------------------------------------------
繰り返し回数: 54
Epoch 1/1
11022/11022 [==============================] - 32s 3ms/step - loss: 0.1764

----diversity 0.2
----- Seedを生成しました:て来ました。
「あ、あいづ専売局だぞ。専売局だぞ。」佐太郎が言いました。
「
て来ました。
「あ、

「ああひう　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　ふうふといっと言ってなんと、きょうからするとゃんなみんながらぎんと言いなしと三郎が、いました。
　はるかどうからなんとから、
「そからにしてやだい。」と言いました。
「するするのだよったりさあからました。
「おうからだい。あるか少し行ぐなぐ。ぐるぐるっこ。」
　それからじしややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややややや

----diversity 1.0
----- Seedを生成しました:まで。」と言って教壇に立ちますと一郎がうしろで、
「気をつけい。」と言いました
まで。」と言って教壇に立ちますと一郎がうしろで、
「気をつけい。」と言いました。
「うん。それからあ、「あ。」一郎かんまる自忘した。の高とみんなさんまになってやっと、へ（って、しました。
「なんだ、あのう、馬にお外で。」らいさないです。」
「それからそれから。ぐ三郎は。」
「おうえないと。」三郎に嘉助をそらにました。
　とうん、もう空たろうとう》なぞ。一をかん行きました。
　どうもうそうはどうからなんだか、へ本をちゃんでして、水を何からどろに日ながら手をもっぽをぐなりました。
　そのうきあいまりは度かく知らない字手をはいてれると、叫びました。
　て一郎、馬のさんかすみんな水をつててて、少した。黒いで三郎が、い馬になかんだか、すぐに水をもはやっときの机の上からどぶに木に、るから三郎《そうだ》の嘉助が、まり息を通どて飛ちゃんとのよいつもして、しながらまたたも「行くなちはこっていました。
　空はうっと《さん》へ行きなくなり）を勢いで三郎が

　なんは

----diversity 1.2
----- Seedを生成しました:まで。」と言って教壇に立ちますと一郎がうしろで、
「気をつけい。」と言いました
まで。」と言って教壇に立ちますと一郎がうしろで、
「気をつけい。」と言いました。
「そだら三けれるがやよろとうちばい粘金と吹いてぢゃ。」二嘉助が嘉助をさんだ見ます。
　それかられを嘉け叫ぶなんが、はえるようからどれをするのぼむと、そのだと